In [1]:
import torch 
import torch.nn as nn
import numpy as np 
import matplotlib.pyplot as plt
import os 
from torch.utils.data import TensorDataset,DataLoader
import re
import nltk
from random import sample
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset,DataLoader
from collections import Counter
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/phaldar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def preprocess(text:str)->list[str]:
    text=text.replace('\n','')
    text=text.split('.')
    text=[item+'.' for item in text]
    return text

def read_data(path:str):
    text=''
    with open(path,'r') as f:
        text=f.read()
        f.close()
    text=preprocess(text)
    return text 
text=read_data('1-18 books combined.txt')
len(text)

17025

In [49]:
sentences = [
    "The cat jumped over the fence.",
    "She went to the market to buy apples.",
    "He loves reading books on rainy days.",
    "They are playing football in the park.",
    "Can you help me with my homework?",
    "The dog barked at the mailman.",
    "We are planning a trip to the mountains.",
    "She smiled and walked away.",
    "He always wakes up early in the morning.",
    "The boy is afraid of the dark.",
    "It was a bright and sunny day.",
    "Why are you late today?",
    "The phone rang loudly in the middle of the night.",
    "She baked a delicious chocolate cake.",
    "He forgot his umbrella at home.",
    "The baby laughed when she saw the toy.",
    "They are waiting for the bus at the station.",
    "She looked at the sky and sighed.",
    "Can you tell me a story?",
    "The children were excited to see the circus.",
    "He wrote a letter to his friend.",
    "She painted a beautiful landscape.",
    "They are learning how to swim.",
    "He was running late for his meeting.",
    "She whispered a secret into his ear.",
    "The teacher asked a difficult question.",
    "The sun was setting behind the hills.",
    "They danced all night at the party.",
    "She opened the door and saw a surprise.",
    "What do you want to eat for dinner?",
    "The stars shone brightly in the sky.",
    "He found an old coin in the garden.",
    "They built a sandcastle on the beach.",
    "She sang a song softly.",
    "The bird flew away from the nest.",
    "He is fixing his bike in the garage.",
    "Do you know the way to the library?",
    "The car stopped suddenly on the road.",
    "She lost her keys somewhere in the house.",
    "They are watching a movie together.",
    "The coffee is too hot to drink.",
    "He gave her a bouquet of flowers.",
    "She wants to become a doctor.",
    "The dog is chasing a ball.",
    "He closed the book and fell asleep.",
    "Are you coming to the party?",
    "She waited patiently for the train.",
    "The snow covered everything in white.",
    "He bought a new pair of shoes.",
    "She hugged her friend tightly.",
    "The wind is blowing through the trees.",
    "They are painting the walls of their house.",
    "He is practicing the guitar every evening.",
    "She walked barefoot on the grass.",
    "The clock struck twelve.",
    "They are setting up the tent for camping.",
    "The dog wagged its tail happily.",
    "She gave him a gift on his birthday.",
    "He told a funny joke to his friends.",
    "They are playing chess in the living room.",
    "The moon is full tonight.",
    "He is writing a story about adventures.",
    "She is learning how to dance.",
    "They went for a walk by the river.",
    "The rain is falling gently.",
    "He made a sandwich for lunch.",
    "She tied her hair in a ponytail.",
    "They are solving a puzzle together.",
    "He cleaned the kitchen after dinner.",
    "She turned off the lights and went to bed.",
    "The flowers are blooming in the garden.",
    "He threw a stone into the water.",
    "She waved goodbye to her friends.",
    "They are playing hide-and-seek.",
    "The bird is building a nest.",
    "He is watching the stars through a telescope.",
    "She is writing a poem about love.",
    "They built a fire in the fireplace.",
    "The kitten is playing with a ball of yarn.",
    "He saw a rainbow in the sky.",
    "She opened the window to let in fresh air.",
    "They are making plans for the weekend.",
    "He helped her carry the groceries.",
    "She listened to music on her headphones.",
    "The leaves are falling from the trees.",
    "He is waiting for the bus to arrive.",
    "She wore a red dress to the party.",
    "They are learning a new language.",
    "He found a treasure map.",
    "She is knitting a scarf for winter.",
    "The waves are crashing on the shore.",
    "He baked bread in the oven.",
    "She read a bedtime story to the children.",
    "They are taking photos of the sunset.",
    "He fixed the broken chair.",
    "She is watering the plants.",
    "The baby is sleeping peacefully.",
    "He drew a picture of a house.",
    "She is feeding the birds in the park.",
    "They are having a picnic under the tree."
]

In [222]:
import re
from collections import Counter

def custom_tokenize(sentence):
    tokens = re.split(r'(\W)', sentence.lower())  # Tokenize words and punctuation
    tokens = [token for token in tokens if token.strip()]  
    return tokens

def tokenize_sentences(sentences, max_len=100):
    tokenized = [custom_tokenize(sentence)[:max_len] for sentence in sentences]
    return tokenized

def build_vocab(tokenized_sentences):
    word_counts = Counter([word for sentence in tokenized_sentences for word in sentence])
    vocab = {word: idx for idx, (word, _) in enumerate(word_counts.items(), start=1)}
    vocab['<PAD>'] = 0  # Add PAD token with index 0
    idx2word = {idx: word for word, idx in vocab.items()}  
    return vocab, idx2word

def sentences_to_sequences(tokenized_sentences, vocab):
    return [[vocab[word] for word in sentence] for sentence in tokenized_sentences]

def pad_sequences(sequences, max_len, pad_type='pre'):
    padded_sequences = []
    for seq in sequences:
        if pad_type == 'post':
            padded = seq + [0] * (max_len - len(seq))
        else:  # Pre-padding
            padded = [0] * (max_len - len(seq)) + seq
        padded_sequences.append(padded[:max_len])  # Ensure max_len
    return padded_sequences


tokenized_sentences = tokenize_sentences(sentences, max_len=10)
vocab, idx2word = build_vocab(tokenized_sentences)

sequences = sentences_to_sequences(tokenized_sentences, vocab)

max_len = 10  # Maximum sequence length
pad_type = 'pre'  # Padding type (pre or post)
padded_sequences = pad_sequences(sequences, max_len, pad_type)

# Example: Convert predicted indices back to words
def decode_sequence(sequence, idx2word):
    return [idx2word[idx] for idx in sequence if idx != 0]  # Ignore PAD tokens

# Test decoding a sequence
decoded_sentence = decode_sequence(padded_sequences[2], idx2word)
print("Decoded Sentence:", ' '.join(decoded_sentence))




Decoded Sentence: he loves reading books on rainy days .


In [53]:


max_len=20
pad_type='pre'

padded_sequences=pad_sequences(sequences,max_len,pad_type)
input_tensor=torch.tensor(padded_sequences,dtype=torch.long)

In [164]:
class StoryLSTM(nn.Module):
    def __init__(self,vocab_size,embed_size,hiddem_size,num_layers):
        super(StoryLSTM,self).__init__()
        #Input: batch_size*seq_len
        self.embedding=nn.Embedding(vocab_size,embed_size)
        #Input: batch_size*seq_len*embed_size
        self.lstm=nn.LSTM(embed_size,hidden_size,num_layers,batch_first=True)
        #Input: batch_size*hidden_size
        self.fc=nn.Linear(hidden_size,vocab_size)

    def forward(self,x,h):
        #Input: batch_size*seq_len
        x=self.embedding(x)
        #Input: batch_size*seq_len*embed_size
        out,(hn,cn)=self.lstm(x,h)
        #Input: batch_size*max_seq_len*hidden_size
        out=self.fc(out)
        return out,hidden

In [165]:
vocab_size=len(vocab)
embed_size=128
hidden_size=256
num_layers=1
model=StoryLSTM(vocab_size,embed_size,hidden_size,num_layers)
model=model.to(device)
model

StoryLSTM(
  (embedding): Embedding(324, 128)
  (lstm): LSTM(128, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=324, bias=True)
)

In [166]:
input_tensor=torch.tensor(padded_sequences,dtype=torch.long)
target_tensor=input_tensor[:,1:]
input_tensor=input_tensor[:,:-1]
batch_size=5
dataset=TensorDataset(input_tensor,target_tensor)
data_loader=DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [167]:
device=torch.device("mps" if torch.backends.mps.is_available() else"cpu")
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001)

In [168]:
num_epochs=5
for epoch in range(num_epochs):
    for inputs,label in data_loader:
        optimizer.zero_grad()
        model.train()
        total_loss=0
    
        hidden=(
            torch.zeros(1,batch_size,256).to(device),
            torch.zeros(1,batch_size,256).to(device)
        )
        inputs=inputs.to(device) 
        label=label.to(device)
        outputs,_=model(inputs,hidden)
        outputs=outputs.view(-1,outputs.size(-1))
        label=label.view(-1)
        loss=ce(outputs,label)
        loss.backward()
        optimizer.step()

In [243]:
preds

['in']

In [254]:
import warnings 
seed_text='They'
warnings.filterwarnings('ignore')
preds=[]
model.eval()
tokens = [vocab.get(word, vocab['<PAD>']) for word in seed_text.lower().split()]
input_seq = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)  # Shape: (1, seq_len)
batch_size=input_seq.size(0) 
hidden=(
    torch.zeros(1,batch_size,hidden_size).to(device),
    torch.zeros(1,batch_size,hidden_size).to(device)
)
genereated_text=seed_text

for _ in range(4):
    with torch.no_grad():
        output,hidden=model(input_seq,hidden)
    probs=torch.softmax(output[:,-1:,:].squeeze(),dim=0)
    preds.append(idx2word[np.argmax(probs.detach().cpu().numpy())])
    req_idx=torch.argmax(probs.squeeze().detach().cpu(),dim=0)
    input_seq=torch.tensor(input_seq.clone().detach()).to(device)
    
print(preds)

['are', 'are', 'are', 'are']


In [192]:
def generate_text(model, seed_text, vocab, idx2word, device, max_len=20):
    model.eval()  # Set the model to evaluation mode

    # Tokenize the seed text
    tokens = [vocab.get(word, vocab['<PAD>']) for word in seed_text.lower().split()]
    input_seq = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)  # Shape: (1, seq_len)

    # Dynamically extract the batch size from input_seq
    batch_size = input_seq.size(0)  # This will be 1 for inference

    # Initialize the hidden state to match the batch size
    hidden = (
        torch.zeros(model.lstm.num_layers, batch_size, model.lstm.hidden_size).to(device),
        torch.zeros(model.lstm.num_layers, batch_size, model.lstm.hidden_size).to(device)
    )

    generated_text = seed_text

    for _ in range(max_len):
        with torch.no_grad():
            # Forward pass through the model
            output, hidden = model(input_seq, hidden)  # Input shape: (1, seq_len)

        # Get the prediction from the last time step
        output = output[:, -1, :]  # Shape: (1, vocab_size)
        probs = torch.softmax(output, dim=-1).cpu().numpy()  # Convert to probabilities

        # Sample the next word index from the probabilities
        next_word_idx = torch.multinomial(torch.tensor(probs), 1).item()

        # Stop if PAD token is predicted
        if next_word_idx == vocab['<PAD>']:
            break

        # Convert the index to a word and append it to the generated text
        next_word = idx2word[next_word_idx]
        generated_text += ' ' + next_word

        # Update the input sequence with the new word
        input_seq = torch.tensor([[next_word_idx]], dtype=torch.long).to(device)

    return generated_text


In [196]:
seed_text="He baked bread"
tokens = [vocab.get(word, vocab['<PAD>']) for word in seed_text.lower().split()]
input_seq = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)  # Shape: (1, seq_len)

In [197]:
input_seq

tensor([[ 13,  70, 302]], device='mps:0')

In [200]:
hidden = (
    torch.zeros(model.lstm.num_layers, 1, model.lstm.hidden_size).to(device),
    torch.zeros(model.lstm.num_layers, 1, model.lstm.hidden_size).to(device)
)

In [212]:
output,_=model(input_seq,hidden)

In [213]:
output.shape

torch.Size([1, 3, 324])

In [204]:
probs = torch.softmax(output, dim=-1).detach().cpu().numpy()  

In [205]:
next_word_idx = torch.multinomial(torch.tensor(probs), 1).item()

RuntimeError: prob_dist must be 1 or 2 dim

In [207]:
idx=np.argmax(probs)

In [220]:
idx=np.argmax(torch.softmax(output[-1][-1],dim=0).detach().cpu().numpy())

In [221]:
idx2word[idx]

'in'

In [228]:
model.eval()
inputs=torch.randint(low=0,high=100,size=(1,5)).to(device)
hidden = (
    torch.zeros(model.lstm.num_layers, 1, model.lstm.hidden_size).to(device),
    torch.zeros(model.lstm.num_layers, 1, model.lstm.hidden_size).to(device)
)
with torch.no_grad():
    op=model(inputs,hidden)

In [230]:
i,hidden=op

In [234]:
hidden[0].size()

torch.Size([1, 1, 256])

In [256]:
import warnings
import numpy as np
import torch

# Suppress warnings
warnings.filterwarnings('ignore')

# Seed text to start generating the sequence
seed_text = 'He'
preds = []

model.eval()  # Set model to evaluation mode

# Convert seed text to tokens and input sequence tensor
tokens = [vocab.get(word, vocab['<PAD>']) for word in seed_text.lower().split()]
input_seq = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)  # Shape: (1, seq_len)

# Initialize the hidden state
batch_size = input_seq.size(0)  # Should be 1 for inference
hidden = (
    torch.zeros(1, batch_size, hidden_size).to(device),
    torch.zeros(1, batch_size, hidden_size).to(device)
)

# Initialize the generated text
generated_text = seed_text

# Generate 4 words sequentially
for _ in range(4):
    with torch.no_grad():
        output, hidden = model(input_seq, hidden)  # Forward pass through the model

    # Get the probabilities of the last time step
    probs = torch.softmax(output[:, -1, :], dim=-1)  # Shape: (1, vocab_size)

    # Get the predicted word index using multinomial sampling for randomness
    next_word_idx = torch.multinomial(probs, 1).item()

    # Convert index to word and add to the generated text
    next_word = idx2word[next_word_idx]
    preds.append(next_word)
    generated_text += ' ' + next_word

    # Update input sequence with the predicted word index
    input_seq = torch.tensor([[next_word_idx]], dtype=torch.long).to(device)

# Print the generated text
print("Generated Text:", generated_text)


Generated Text: He letter keys fixed after


In [261]:
import warnings
import numpy as np
import torch

def generate_sequence(model, seed_text, vocab, idx2word, device, hidden_size, num_words=4):
    """
    Generate a sequence of text based on a seed input.

    Args:
    - model: Trained LSTM model for text generation.
    - seed_text: Initial text to start generating from.
    - vocab: Dictionary mapping words to their token indices.
    - idx2word: Dictionary mapping token indices back to words.
    - device: Device (CPU or GPU) for tensor computations.
    - hidden_size: Size of hidden state in the LSTM.
    - num_words: Number of words to generate.

    Returns:
    - generated_text: Complete text generated from the model.
    """
    # Suppress warnings
    warnings.filterwarnings('ignore')

    model.eval()  # Set model to evaluation mode

    # Convert seed text to tokens and input sequence tensor
    tokens = [vocab.get(word, vocab['<PAD>']) for word in seed_text.lower().split()]
    input_seq = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)  # Shape: (1, seq_len)

    # Initialize hidden state
    batch_size = input_seq.size(0)
    hidden = (
        torch.zeros(1, batch_size, hidden_size).to(device),
        torch.zeros(1, batch_size, hidden_size).to(device)
    )

    generated_text = seed_text  # Start with seed text
    preds = []

    # Generate the specified number of words
    for _ in range(num_words):
        with torch.no_grad():
            output, hidden = model(input_seq, hidden)

        # Get probabilities from the last time step
        probs = torch.softmax(output[:, -1, :], dim=-1)

        # Sample the next word index
        next_word_idx = torch.multinomial(probs, 1).item()

        # Stop if PAD token is generated
        if next_word_idx == vocab['<PAD>']:
            break

        # Get the word and add it to the text
        next_word = idx2word[next_word_idx]
        preds.append(next_word)
        generated_text += ' ' + next_word

        # Update input sequence with the predicted word
        input_seq = torch.tensor([[next_word_idx]], dtype=torch.long).to(device)

    return generated_text

# Example usage
generated_text = generate_sequence(
    model=model,
    seed_text='He',
    vocab=vocab,
    idx2word=idx2word,
    device=device,
    hidden_size=256,
    num_words=4
)

print("Generated Text:", generated_text)


Generated Text: He falling weekend night market
